In [59]:
!pip install comet_ml
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
!pip install lightning
import lightning
from lightning.fabric import Fabric
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import re
import os
import lzma
from tqdm import tqdm
import mmap
import random
import matplotlib.pyplot as plt
import numpy as np

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
# fabric = Fabric(precision="16-mixed")
# device = fabric.device
device = torch.device("cuda:1")

# Data Preprocessing

In [61]:
# Redefining the clean_text function
def clean_text(text):
    # Lowercase the text
    text = text.lower()
    # print(text, ' p1')
    # Remove words that contain numbers
    text = re.sub(r'\w*\d\w*', '', text)

    # remove digits longer than 5
    text = re.sub(r'\d{5,}', '', text)

    # remove words longer than 20
    text = re.sub(r'\b\w{21,}\b', '', text)

    # Preserve meaningful punctuation and numbers
    text = re.sub(r"[^a-z0-9.,!?;\s']", '', text)

    # Remove words with characters other than letters in them
    text = re.sub(r'\s\w+[.,!?;]\w+\s', ' ', text)

    words = text.split()
    text = ' '.join(words)

    # Truncate text to start after the first period and end at the last period
    start = text.find('.') + 1
    end = text.rfind('.')
    if start != 0 and end != -1 and end > start:
        text = text[start:end].strip()

    text = re.sub(r'\,+[,\s]+[^\w]', ', ', text) # Remove occurences such as , ,
    text = re.sub(r'\.+[\.\s]+[^\w]', '. ', text) # Remove occurences such as . .

    text = re.sub(r'\s+\.', '.' , text) # If there is space before '.' remove it
    text = re.sub(r'\s+[,]', ',' , text) # If there is space before ',' remove it
    text = re.sub(r'\s+[!]', '!' , text) # If there is space before '!' remove it
    text = re.sub(r'\s+[?]', '?' , text) # If there is space before '?' remove it

    # Remove unnecessary whitespaces and handle line breaks
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    # Remove repeating commas and periods
    text = re.sub(r'[.]+', '.', text)
    text = re.sub(r'[,]+', ',', text)
    text = re.sub(r'\.+[,]+', ',', text)

    return text

def is_valid_line(line):
  if line:

    if len(line) < 2:
      return False

    if not re.search('[a-z]', line):
      return False

    return True

  return False

In [62]:
from transformers import AutoTokenizer

# For instance, using the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = len(tokenizer)

In [63]:
# #loading the vocab associated with mini pile
# vocab_file_name = "/kaggle/input/mini-pile/mini_pile_train_vocab.txt"
# with open(vocab_file_name, 'r' , encoding='utf-8') as f:
#   chars = sorted(list(set(f.read())))
#   f.close

# Tokenize

## Character level encoding

In [64]:
# stoi = { ch:i for i, ch in enumerate(chars)}
# itos = { i:ch for i, ch in enumerate(chars)}

# # encoder: string to int
# encode = lambda s: [stoi[c] for c in s]

# # decoder: int to string
# decode = lambda l: ''.join([itos[i] for i in l])

# Model

In [65]:
# single head
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v
    return out

# multi-head
class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

  '''
  # post norm
  def forward(self, x):
    y = self.sa(x)
    x = self.ln1(x + y)
    y = self.ffwd(x)
    x = self.ln2(x + y)
    return x
  '''

In [66]:
class GPTLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, loss = self(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

# Batches

In [67]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    
    if split == 'train': 
        filename = "/kaggle/input/tiny-stories/tiny_stores_train.txt" 
    else: 
        filename = "/kaggle/input/tiny-stories/tiny_stores_val.txt" 
        
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # if the data is not long enough
            while len(block) - block_size <= 0:
              block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            # clean the data
            decoded_block = clean_text(decoded_block)

            # Train and test splits
            data = torch.tensor(tokenizer.encode(decoded_block), dtype=torch.long)

    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [68]:
# # data loading
# def get_batch(split):
#     # generate a small batch of data of inputs x and targets y
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x, y = x.to(device), y.to(device)
#     return x, y

In [69]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.mean().item()

        out[split] = losses.mean()
    model.train()
    return out

# Training

## Hyperparameters

In [70]:
# hyperparameters
batch_size = 128
block_size = 128
max_iters = 2000
eval_interval = 100
learning_rate = 3e-2
eval_iters = 2000
n_embd = 200
n_head = 4
n_layer = 4
dropout = 0.0
# device = fabric.device
device = torch.device("cuda:1")
# vocab_size = len(chars)
vocab_size = len(tokenizer)

hyperparameters = {
    'batch_size': batch_size,
    'block_size': block_size,
    'max_iters': max_iters,
    'eval_interval': eval_interval,
    'learning_rate': learning_rate,
    'eval_iters': eval_iters,
    'n_embd': n_embd,
    'n_head': n_head,
    'n_layer': n_layer,
    'dropout': dropout,
}

# Hyperparameter -> Loss Visualization Data
# Essentially for every run, record the hyperparams and
# Plot them compared to old (save old in file)
vis_data = {
    'batch_size': batch_size,
    'block_size': block_size,
    'max_iters': max_iters,
    'eval_interval': eval_interval,
    'learning_rate': learning_rate,
    'eval_iters': eval_iters,
    'n_embd': n_embd,
    'n_head': n_head,
    'n_layer': n_layer,
    'dropout': dropout,
}

vis_data_file_name = "visual_data_during_training.txt"
vis_post_data_file_name = "visual_data_post_training.txt"

In [71]:
# model
model = GPTLanguageModel()
model.to(device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# Visualization Data
vis_data_iterations = [] # Format: [[iteration, train_loss, val_loss]]

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

14.193322 M parameters


In [72]:
# paths
base_model_200_4_4_path = '/kaggle/input/base-models/base_model_200_4_4.pth'

In [73]:
# load the model and optimizer
path = '/kaggle/input/base-models/base_model_200_4_4.pth'
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


In [74]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Dec  2 05:08:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   4646MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [75]:
import torch.optim as optim

experiment = Experiment(
  api_key="78u2AfbhkXeTChB3Kzb7FhOEY",
  project_name="JokeGPT",
  workspace="lzh0212"
)

experiment.log_parameters(hyperparameters)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

losses = 0
for iter in range(max_iters):

    xb, yb = get_batch('train')

    # evaluate
    logits, loss = model(xb, yb)
    losses += loss.item()  # Use .item() to get the scalar value
    
    # Log and print at evaluation intervals
    if iter % eval_interval == 0 or iter == max_iters - 1:
        avg_loss = losses / (iter + 1)
        current_lr = scheduler.get_last_lr()[0]  # Get current learning rate

        print(f"Iteration {iter}: Avg Loss = {avg_loss:.4f}, LR = {current_lr}")

        # Log metrics to experiment
        experiment.log_metric("avg_loss", avg_loss, step=iter)
        experiment.log_metric("learning_rate", current_lr, step=iter)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    scheduler.step()

experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/lzh0212/jokegpt/bdea953e876240e98b9552ffaeaaf47f
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 128
COMET INFO:     block_size    : 128
COMET INFO:     dropout       : 0.0
COMET INFO:     eval_interval : 100
COMET INFO:     eval_iters    : 2000
COMET INFO:     learning_rate : 0.03
COMET INFO:     max_iters     : 2000
COMET INFO:     n_embd        : 200
COMET INFO:     n_head        : 4
COMET INFO:     n_layer       : 4
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment deta

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.86 GiB (GPU 1; 14.76 GiB total capacity; 13.39 GiB already allocated; 481.75 MiB free; 13.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Visualization

# Save the model

In [ ]:
def save(model, optimizer, hyperparameters, base_path='/kaggle/working/'):
    n_embd = hyperparameters['n_embd']
    n_head = hyperparameters['n_head']
    n_layer = hyperparameters['n_layer']

    filename = f"base_model_{n_embd}_{n_head}_{n_layer}.pth"
    full_path = base_path + filename

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'hyperparameters': hyperparameters
    }, full_path)

    print(f"Checkpoint saved to {full_path}")

In [ ]:
save(model, optimizer, hyperparameters)

In [ ]:
checkpoint['hyperparameters']

# Result

In [ ]:
prompt = 'hello'
model = model.module
model.to(device)
context = torch.tensor(tokenizer.encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=200)[0].tolist()))

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/lzh0212/jokegpt/6590c2743e9943829bb32e51a20af047
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 128
COMET INFO:     block_size    : 128
COMET INFO:     dropout       : 0.0
COMET INFO:     eval_interval : 100
COMET INFO:     eval_iters    : 2000
COMET INFO:     learning_rate : 0.03
COMET INFO:     max_iters     : 2000
COMET INFO:     n_embd        : 200
COMET INFO:     n_head        : 4
COMET INFO:     n_layer       : 4
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment deta